# Alignment Code for DTW

In [78]:
%matplotlib inline
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [79]:
import numpy as np
import matplotlib.pyplot as plt
import librosa as lb
import os.path
from pathlib import Path
import pickle
import multiprocessing
import time
import gc
import math
import import_ipynb

## Cumulative Cost, Steps and Path

In [3]:
%%cython
import numpy as np
cimport numpy as np
cimport cython

import sys
import time

DTYPE_INT32 = np.int32
ctypedef np.int32_t DTYPE_INT32_t

DTYPE_FLOAT = np.float64
ctypedef np.float64_t DTYPE_FLOAT_t

cdef DTYPE_FLOAT_t MAX_FLOAT = float('inf')

# careful, without bounds checking can mess up memory - also can't use negative indices I think (like x[-1])
@cython.boundscheck(False) # turn off bounds-checking for entire function
def DTW_Cost_To_AccumCostAndSteps(Cin, parameter, WarpMax=None):
    '''
    Inputs
        C: The cost Matrix
    '''

    '''
    Section for checking and catching errors in the inputs
    '''

    cdef np.ndarray[DTYPE_FLOAT_t, ndim=2] C
    try:
        C = np.array(Cin, dtype=DTYPE_FLOAT)
    except TypeError:
        print(bcolors.FAIL + "FAILURE: The type of the cost matrix is wrong - please pass in a 2-d numpy array" + bcolors.ENDC)
        return [-1, -1, -1]
    except ValueError:
        print(bcolors.FAIL + "FAILURE: The type of the elements in the cost matrix is wrong - please have each element be a float (perhaps you passed in a matrix of ints?)" + bcolors.ENDC)
        return [-1, -1, -1]

    cdef np.ndarray[np.uint32_t, ndim=1] dn
    cdef np.ndarray[np.uint32_t, ndim=1] dm
    cdef np.ndarray[DTYPE_FLOAT_t, ndim=1] dw
    # make sure dn, dm, and dw are setup
    # dn loading and exception handling
    if ('dn' in parameter.keys()):
        try:
            dn = np.array(parameter['dn'], dtype=np.uint32)
        except TypeError:
            print(bcolors.FAIL + "FAILURE: The type of dn (row steps) is wrong - please pass in a 1-d numpy array that holds uint32s" + bcolors.ENDC)
            return [-1, -1, -1]
        except ValueError:
            print(bcolors.FAIL + "The type of the elements in dn (row steps) is wrong - please have each element be a uint32 (perhaps you passed a long?). You can specify this when making a numpy array like: np.array([1,2,3],dtype=np.uint32)" + bcolors.ENDC)
            return [-1, -1, -1]
    else:
        dn = np.array([1, 1, 0], dtype=np.uint32)
    # dm loading and exception handling
    if 'dm'  in parameter.keys():
        try:
            dm = np.array(parameter['dm'], dtype=np.uint32)
        except TypeError:
            print(bcolors.FAIL + "FAILURE: The type of dm (col steps) is wrong - please pass in a 1-d numpy array that holds uint32s" + bcolors.ENDC)
            return [-1, -1, -1]
        except ValueError:
            print(bcolors.FAIL + "FAILURE: The type of the elements in dm (col steps) is wrong - please have each element be a uint32 (perhaps you passed a long?). You can specify this when making a numpy array like: np.array([1,2,3],dtype=np.uint32)" + bcolors.ENDC)
            return [-1, -1, -1]
    else:
        print(bcolors.FAIL + "dm (col steps) was not passed in (gave default value [1,0,1]) " + bcolors.ENDC)
        dm = np.array([1, 0, 1], dtype=np.uint32)
    # dw loading and exception handling
    if 'dw'  in parameter.keys():
        try:
            dw = np.array(parameter['dw'], dtype=DTYPE_FLOAT)
        except TypeError:
            print(bcolors.FAIL + "FAILURE: The type of dw (step weights) is wrong - please pass in a 1-d numpy array that holds floats" + bcolors.ENDC)
            return [-1, -1, -1]
        except ValueError:
            print(bcolors.FAIL + "FAILURE:The type of the elements in dw (step weights) is wrong - please have each element be a float (perhaps you passed ints or a long?). You can specify this when making a numpy array like: np.array([1,2,3],dtype=np.float64)" + bcolors.ENDC)
            return [-1, -1, -1]
    else:
        dw = np.array([1, 1, 1], dtype=DTYPE_FLOAT)
        print(bcolors.FAIL + "dw (step weights) was not passed in (gave default value [1,1,1]) " + bcolors.ENDC)
    
    '''
    Section where types are given to the variables we're going to use 
    '''
    # create matrices to store our results (D and E)
    cdef DTYPE_INT32_t numRows = C.shape[0] # only works with np arrays, use np.shape(x) will work on lists? want to force to use np though?
    cdef DTYPE_INT32_t numCols = C.shape[1]
    cdef DTYPE_INT32_t numDifSteps = np.size(dw)

    cdef unsigned int maxRowStep = max(dn)
    cdef unsigned int maxColStep = max(dm)

    cdef np.ndarray[np.uint32_t, ndim=2] steps = np.zeros((numRows,numCols), dtype=np.uint32)
    cdef np.ndarray[DTYPE_FLOAT_t, ndim=2] accumCost = np.ones((maxRowStep + numRows, maxColStep + numCols), dtype=DTYPE_FLOAT) * MAX_FLOAT

    cdef DTYPE_FLOAT_t bestCost
    cdef DTYPE_INT32_t bestCostIndex
    cdef DTYPE_FLOAT_t costForStep
    cdef unsigned int row, col
    cdef unsigned int stepIndex

    '''
    The start of the actual algorithm, now that all our variables are set up
    '''
    # initializing the cost matrix - depends on whether its subsequence DTW
    # essentially allow us to hop on the bottom anywhere (so could start partway through one of the signals)
    if parameter['SubSequence']:
        for col in range(numCols):
            accumCost[maxRowStep, col + maxColStep] = C[0, col]
    else:
        accumCost[maxRowStep, maxColStep] = C[0,0]

    # ADDED
    remove_x = False
    remove_y = False

    # filling the accumulated cost matrix
    for row in range(maxRowStep, numRows + maxRowStep, 1):
        if WarpMax and row % WarpMax == 0 and row > 1: # ADDED
            remove_x = True

        for col in range(maxColStep, numCols + maxColStep, 1):
            if WarpMax and col % WarpMax == 0: # ADDED
                remove_y = True
            
            bestCost = accumCost[<unsigned int>row, <unsigned int>col] # initialize with what's there - so if is an entry point, then can start low
            bestCostIndex = 0
            # go through each step, find the best one
            for stepIndex in range(numDifSteps):
                if remove_x and dn[(stepIndex)]==1 and dm[(stepIndex)]==0: # ADDED
                    continue
                if remove_y and dm[(stepIndex)]==1 and dn[(stepIndex)]==0:
                    continue
                costForStep = accumCost[<unsigned int>((row - dn[(stepIndex)])), <unsigned int>((col - dm[(stepIndex)]))] + dw[stepIndex] * C[<unsigned int>(row - maxRowStep), <unsigned int>(col - maxColStep)]
                if costForStep < bestCost:
                    bestCost = costForStep
                    bestCostIndex = stepIndex
            
            # save the best cost and best cost index
            accumCost[row, col] = bestCost
            steps[<unsigned int>(row - maxRowStep), <unsigned int>(col - maxColStep)] = bestCostIndex
            remove_y = False
        remove_x = False

    # return the accumulated cost along with the matrix of steps taken to achieve that cost
    return [accumCost[maxRowStep:, maxColStep:], steps]

@cython.boundscheck(False) # turn off bounds-checking for entire function

def DTW_GetPath(np.ndarray[DTYPE_FLOAT_t, ndim=2] accumCost, np.ndarray[np.uint32_t, ndim=2] stepsForCost, parameter):
    '''
    Parameter should have: 'dn', 'dm', 'dw', 'SubSequence'
    '''

    cdef np.ndarray[unsigned int, ndim=1] dn
    cdef np.ndarray[unsigned int, ndim=1] dm
    cdef np.uint8_t subseq
    cdef np.int32_t startCol # added
    # make sure dn, dm, and dw are setup
    if ('dn'  in parameter.keys()):
        dn = parameter['dn']
    else:
        dn = np.array([1, 1, 0], dtype=DTYPE_INT32)
    if 'dm'  in parameter.keys():
        dm = parameter['dm']
    else:
        dm = np.array([1, 0, 1], dtype=DTYPE_INT32)
    if 'SubSequence' in parameter.keys():
        subseq = parameter['SubSequence']
    else:
        subseq = 0
    
    # added START
    if 'startCol' in parameter.keys(): 
        startCol = parameter['startCol']
    else:
        startCol = -1
    # added END

    cdef np.uint32_t numRows
    cdef np.uint32_t numCols
    cdef np.uint32_t curRow
    cdef np.uint32_t curCol
    cdef np.uint32_t endCol
    cdef DTYPE_FLOAT_t endCost

    numRows = accumCost.shape[0]
    numCols = accumCost.shape[1]

    # either start at the far corner (non sub-sequence)
    # or start at the lowest cost entry in the last row (sub-sequence)
    # where all of the signal along the row has been used, but only a 
    # sub-sequence of the signal along the columns has to be used
    curRow = numRows - 1
    if subseq:
        curCol = np.argmin(accumCost[numRows - 1, :])
    else:
        curCol = numCols - 1
        
    # added - if specified, overrides above
    if startCol >= 0:
        curCol = startCol

    endCol = curCol
    endCost = accumCost[curRow, curCol]

    cdef np.uint32_t curRowStep
    cdef np.uint32_t curColStep
    cdef np.uint32_t curStepIndex

    cdef np.ndarray[np.uint32_t, ndim=2] path = np.zeros((2, numRows + numCols), dtype=np.uint32) # make as large as could need, then chop at the end
    path[0, 0] = curRow
    path[1, 0] = curCol

    cdef np.uint32_t stepsInPath = 1 # starts at one, we add in one before looping
    cdef np.uint32_t stepIndex = 0
    cdef np.int8_t done = (subseq and curRow == 0) or (curRow == 0 and curCol == 0)
    while not done:
        if accumCost[curRow, curCol] == MAX_FLOAT:
            print('A path is not possible')
            break

        # you're done if you've made it to the bottom left (non sub-sequence)
        # or just the bottom (sub-sequence)
        # find the step size
        curStepIndex = stepsForCost[curRow, curCol]
        curRowStep = dn[curStepIndex]
        curColStep = dm[curStepIndex]
        # backtrack by 1 step
        curRow = curRow - curRowStep
        curCol = curCol - curColStep
        # add your new location onto the path
        path[0, stepsInPath] = curRow
        path[1, stepsInPath] = curCol
        stepsInPath = stepsInPath + 1
        # check to see if you're done
        done = (subseq and curRow == 0) or (curRow == 0 and curCol == 0)

    # reverse the path (a matrix with two rows) and return it
    return [np.fliplr(path[:, 0:stepsInPath]), endCol, endCost]

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

## AlignDTW

In [4]:
def alignDTW(featfile1, featfile2, steps, weights, method, WarpMax, outfile, profile):
    
    F1 = np.load(featfile1) # 12 x N
    F2 = np.load(featfile2) # 12 x M
    # we assume that N >= M 
    N, M = F1.shape[1], F2.shape[1]
    times = []
    times.append(time.time())
    if method == "downsampleQuantized":
        # we wish to only select M columns of of F1 to get (12 x M)
        index = [int(round(x)) for x in np.linspace(0, N-1, M)]
        F1 = F1[:, index]
    elif method == "downsampleInterpolate":
        # we want to multiply matrix (12 x N) by (N x M) to get (12 x M)
        transform = np.zeros((N, M))
        index = np.linspace(0, N-1, M) # M indices evenly spaced between [0, N-1]
        for col, index in enumerate(index):
            # at column m, insert weight RIGHT at position ROW and LEFT at position ROW+1
            row = int(index)
            right = index - int(index)
            left = 1 - right
            # if we are at the last row, insert weight 1
            if row + 1 == N:
                transform[row, col] = 1
                continue
            transform[row, col] = left
            transform[row+1, col] = right
        F1 = F1 @ transform
    elif method == "upsampleQuantized":
        index = [int(x) for x in np.linspace(0, M-1, N)]
        F2 = F2[:, index] 
    elif method == "upsampleInterpolate":
        # we want to multiply matrix (12 x M) by (M x N) to get (12 x N)
        transform = np.zeros((M, N))
        index = np.linspace(0, M-1, N) # N indices evenly spaced between [0, M-1]
        for col, index in enumerate(index):
            # at column N, insert weight RIGHT at position ROW and LEFT at position ROW+1
            row = int(index)
            right = index - int(index)
            left = 1 - right
            # if we are at the last row, insert weight 1
            if row + 1 == M:
                transform[row, col] = 1
                continue
            transform[row, col] = left
            transform[row+1, col] = right
        F2 = F2 @ transform
    elif method == "adaptiveWeight1":
        weights = np.array([N/M, 1])
    elif method == "adaptiveWeight2":
        weights = np.array([N/M, 1, 1 + N/M])
    
    C = 1 - F1.T @ F2 # cost distance metric
    times.append(time.time())

    dn = steps[:,0].astype(np.uint32)
    dm = steps[:,1].astype(np.uint32)
    parameters = {'dn': dn, 'dm': dm, 'dw': weights, 'SubSequence': False}
    [D, s] = DTW_Cost_To_AccumCostAndSteps(C, parameters, WarpMax)
    times.append(time.time())
    [wp, endCol, endCost] = DTW_GetPath(D, s, parameters)
    times.append(time.time())

    if method == "downsampleQuantized" or method == "downsampleInterpolate":
        wp[0] = wp[0] * N / M
    elif method == "upsampleQuantized" or method == "upsampleInterpolate":
        wp[1] = wp[1] * M / N

    if outfile:
        pickle.dump(wp, open(outfile, 'wb'))
        
    # profile will mark times [before_accum_cost, after_accum_cost, after_path]
    return wp, np.diff(times) if profile else wp

In [5]:
def alignDTW_batch(querylist, featdir1, featdir2, outdir, n_cores, steps, weights, method, warpMax, profile):
    outdir.mkdir(parents=True, exist_ok=True)
    inputs = []
    with open(querylist, 'r') as f:
        for line in f:
            parts = line.strip().split(' ')
            assert len(parts) == 2
            featfile1 = (featdir1 / parts[0]).with_suffix('.npy')
            featfile2 = (featdir2 / parts[1]).with_suffix('.npy')
            queryid = os.path.basename(parts[0]) + '__' + os.path.basename(parts[1])
            outfile = (outdir / queryid).with_suffix('.pkl')
            if os.path.exists(outfile):
                print(f"Skipping {outfile}")
            inputs.append((featfile1, featfile2, steps, weights, method, warpMax, outfile, profile))
    # process files in parallel
    pool = multiprocessing.Pool(processes = n_cores)
    pool.starmap(alignDTW, inputs)
    return

##### Single working example

In [ ]:
# dir1 = Path('Mazurkas_median_x2.000///features/clean/Chopin_Op017No4/Chopin_Op017No4_Magaloff-1977_pid5667267b-10.npy')
# dir2 = Path('Mazurkas_median_x0.630/features/clean/Chopin_Op017No4/Chopin_Op017No4_Fou-1978_pid9071-20.npy')
# F1 = np.load(dir1) # 12 x N
# F2 = np.load(dir2) # 12 x M
# N, M = F1.shape[1], F2.shape[1]

##### Testing each method

In [ ]:
# n_cores, steps, weights, downsample, alignment_function = get_DTW_settings('DTW1')
# dir1 = Path('Mazurkas_median_x2.000///features/clean/Chopin_Op017No4/Chopin_Op017No4_Magaloff-1977_pid5667267b-10.npy')
# dir2 = Path('Mazurkas_median_x0.630/features/clean/Chopin_Op017No4/Chopin_Op017No4_Fou-1978_pid9071-20.npy')
# outfile = Path("experiments_test/train_toy.x2000.x0630.DTW/Chopin_Op017No4_Magaloff-1977_pid5667267b-10__Chopin_Op017No4_Fou-1978_pid9071-20.pkl")
# alignDTW(dir1, dir2, steps, weights, "downsampleQuantized", None, outfile, True)

## Get Settings

In [6]:
def get_settings(algorithm):
    """
    Given an algorithm, 
        Return n_cores, steps, weights, method, warpMax
    """
    n_cores = 40 # multiprocessing.cpu_count()
    steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
    weights = np.array([2,3,3])
    method = "DTW"
    warpMax = None

    if algorithm == 'DTW2':
        steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
        weights = np.array([1,2,2])
    elif algorithm == 'DTW3':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        weights = np.array([1,1,2])
    elif algorithm == 'DTW4':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        weights = np.array([1,1,1])
    elif algorithm == 'DTW5':
        steps = np.array([0,1,1,0]).reshape((-1,2))
        weights = np.array([1,1])
    elif algorithm == 'DTW1_add3':
        steps = np.array([1,1,1,2,2,1,1,3,3,1]).reshape((-1,2))
        weights = np.array([2,3,3,4,4])
    elif algorithm == 'DTW1_add4':
        steps = np.array([1,1,1,2,2,1,1,3,3,1,1,4,4,1]).reshape((-1,2))
        weights = np.array([2,3,3,4,4,5,5])
    elif algorithm == 'DTW1_downsampleQuantized':
        method = "downsampleQuantized"
    elif algorithm == 'DTW1_downsampleInterpolate':
        method = "downsampleInterpolate"
    elif algorithm == 'DTW1_upsampleQuantized':
        method = "upsampleQuantized"
    elif algorithm == 'DTW1_upsampleInterpolate':
        method = "upsampleInterpolate"
    elif algorithm == 'DTW_adaptiveWeight1':
        steps = np.array([0,1,1,0]).reshape((-1,2))
        method = "adaptiveWeight1"
    elif algorithm == 'DTW_adaptiveWeight2':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        method = "adaptiveWeight2"
    elif algorithm == 'DTW_selectiveTransitions':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        weights = np.array([1,1,2])
        warpMax = 3
        
    return n_cores, steps, weights, method, warpMax


## Run Benchmarks

In [7]:
def single_benchmark(filelist, timing1, timing2, algorithm, profile, outdir='experiments_test'):
    """
    Inputs: 'train_toy', 'Mazurkas_median_x1.000', 'Mazurkas_median_x1.000', 'DTW1'
    Output: Run specified DTW algorithm for a single system description
    """
    query_list = 'cfg_files/filelist.{}.txt'.format(filelist) # make 'train_toy' to actual file path --> 'cfg_files/filelist.train_toy.txt'
    featdir1 = Path(f"Mazurkas_median_{timing1}/features/clean")
    featdir2 = Path(f"Mazurkas_median_{timing2}/features/clean")
    outdir = Path(f'{outdir}/{filelist}.{timing1}.{timing2}.{algorithm}')
    # get alignment settings based on specified alignment type
    n_cores, steps, weights, method, warpMax = get_settings(algorithm)
    alignDTW_batch(query_list, featdir1, featdir2, outdir, n_cores, steps, weights, method, warpMax, profile)
    return

In [8]:
def multiple_benchmark(settings, profile = None):
    """
    Settings is a list of individual system descriptions to be run.
        ie. [['train_toy', 'Mazurkas_median_x1.000', 'Mazurkas_median_x1.000', 'DTW1'],
             ['train_toy', 'Mazurkas_median_x1.260', 'Mazurkas_median_x1.000', 'DTW1'],...] 
    """
    for setting in settings:
        filelist, timing1, timing2, algorithm = setting
        single_benchmark(filelist, timing1, timing2, algorithm, profile)
        print(f"Finished alignment for: {filelist} {timing1} {timing2} {algorithm}")
    return

## Run test

In [9]:
def get_arguments(filelist, algorithms):
    '''create a list of lists to pass into multiple_benchmark based on what alignments we want'''
    res = []
    for algorithm in algorithms:
        res += [[filelist, 'x1.000', 'x1.000', algorithm],
                [filelist, 'x1.260', 'x1.000', algorithm],
                [filelist, 'x1.260', 'x0.794', algorithm],
                [filelist, 'x1.588', 'x0.794', algorithm],
                [filelist, 'x1.588', 'x0.630', algorithm],
                [filelist, 'x2.000', 'x0.630', algorithm],
                [filelist, 'x2.000', 'x0.500', algorithm]]
    return res

In [11]:
DTW_normal = get_arguments('train_toy', ['DTW1', 'DTW2', 'DTW3', 'DTW4', 'DTW5'])
multiple_benchmark(DTW_normal)

Finished alignment for: train_toy x1.000 x1.000 DTW1
Finished alignment for: train_toy x1.260 x1.000 DTW1
Finished alignment for: train_toy x1.260 x0.794 DTW1
Finished alignment for: train_toy x1.588 x0.794 DTW1
A path is not possible
A path is not possible
A path is not possible
A path is not possible
A path is not possible
Finished alignment for: train_toy x1.588 x0.630 DTW1
A path is not possible
A path is not possible
A path is not possible
A path is not possible
A path is not possible
Finished alignment for: train_toy x2.000 x0.630 DTW1
A path is not possibleA path is not possible

A path is not possible
A path is not possible
A path is not possible
Finished alignment for: train_toy x2.000 x0.500 DTW1
Finished alignment for: train_toy x1.000 x1.000 DTW2
Finished alignment for: train_toy x1.260 x1.000 DTW2
Finished alignment for: train_toy x1.260 x0.794 DTW2
Finished alignment for: train_toy x1.588 x0.794 DTW2
A path is not possible
A path is not possible
A path is not possible
A p

In [12]:
DTW_adds = get_arguments('train_toy', ['DTW1_add3', 'DTW1_add4'])
multiple_benchmark(DTW_adds)

Finished alignment for: train_toy x1.000 x1.000 DTW1_add3
Finished alignment for: train_toy x1.260 x1.000 DTW1_add3
Finished alignment for: train_toy x1.260 x0.794 DTW1_add3
Finished alignment for: train_toy x1.588 x0.794 DTW1_add3
Finished alignment for: train_toy x1.588 x0.630 DTW1_add3
A path is not possible
A path is not possible
A path is not possible
A path is not possible
A path is not possible
Finished alignment for: train_toy x2.000 x0.630 DTW1_add3
A path is not possible
A path is not possible
A path is not possible
A path is not possible
A path is not possible
Finished alignment for: train_toy x2.000 x0.500 DTW1_add3
Finished alignment for: train_toy x1.000 x1.000 DTW1_add4
Finished alignment for: train_toy x1.260 x1.000 DTW1_add4
Finished alignment for: train_toy x1.260 x0.794 DTW1_add4
Finished alignment for: train_toy x1.588 x0.794 DTW1_add4
Finished alignment for: train_toy x1.588 x0.630 DTW1_add4
Finished alignment for: train_toy x2.000 x0.630 DTW1_add4
Finished alignme

In [13]:
DTW_samples = get_arguments('train_toy', ['DTW1_downsampleQuantized', 'DTW1_downsampleInterpolate', 'DTW1_upsampleQuantized', 'DTW1_upsampleInterpolate'])
multiple_benchmark(DTW_samples)

Finished alignment for: train_toy x1.000 x1.000 DTW1_downsampleQuantized
Finished alignment for: train_toy x1.260 x1.000 DTW1_downsampleQuantized
Finished alignment for: train_toy x1.260 x0.794 DTW1_downsampleQuantized
Finished alignment for: train_toy x1.588 x0.794 DTW1_downsampleQuantized
Finished alignment for: train_toy x1.588 x0.630 DTW1_downsampleQuantized
Finished alignment for: train_toy x2.000 x0.630 DTW1_downsampleQuantized
Finished alignment for: train_toy x2.000 x0.500 DTW1_downsampleQuantized
Finished alignment for: train_toy x1.000 x1.000 DTW1_downsampleInterpolate
Finished alignment for: train_toy x1.260 x1.000 DTW1_downsampleInterpolate
Finished alignment for: train_toy x1.260 x0.794 DTW1_downsampleInterpolate
Finished alignment for: train_toy x1.588 x0.794 DTW1_downsampleInterpolate
Finished alignment for: train_toy x1.588 x0.630 DTW1_downsampleInterpolate
Finished alignment for: train_toy x2.000 x0.630 DTW1_downsampleInterpolate
Finished alignment for: train_toy x2.00

In [14]:
DTW_weights = get_arguments('train_toy', ['DTW_adaptiveWeight1', 'DTW_adaptiveWeight2'])
multiple_benchmark(DTW_weights)

Finished alignment for: train_toy x1.000 x1.000 DTW_adaptiveWeight1
Finished alignment for: train_toy x1.260 x1.000 DTW_adaptiveWeight1
Finished alignment for: train_toy x1.260 x0.794 DTW_adaptiveWeight1
Finished alignment for: train_toy x1.588 x0.794 DTW_adaptiveWeight1
Finished alignment for: train_toy x1.588 x0.630 DTW_adaptiveWeight1
Finished alignment for: train_toy x2.000 x0.630 DTW_adaptiveWeight1
Finished alignment for: train_toy x2.000 x0.500 DTW_adaptiveWeight1
Finished alignment for: train_toy x1.000 x1.000 DTW_adaptiveWeight2
Finished alignment for: train_toy x1.260 x1.000 DTW_adaptiveWeight2
Finished alignment for: train_toy x1.260 x0.794 DTW_adaptiveWeight2
Finished alignment for: train_toy x1.588 x0.794 DTW_adaptiveWeight2
Finished alignment for: train_toy x1.588 x0.630 DTW_adaptiveWeight2
Finished alignment for: train_toy x2.000 x0.630 DTW_adaptiveWeight2
Finished alignment for: train_toy x2.000 x0.500 DTW_adaptiveWeight2


In [15]:
DTW_selective = get_arguments('train_toy', ['DTW_selectiveTransitions'])
multiple_benchmark(DTW_selective)

Finished alignment for: train_toy x1.000 x1.000 DTW_selectiveTransitions
Finished alignment for: train_toy x1.260 x1.000 DTW_selectiveTransitions
Finished alignment for: train_toy x1.260 x0.794 DTW_selectiveTransitions
Finished alignment for: train_toy x1.588 x0.794 DTW_selectiveTransitions
Finished alignment for: train_toy x1.588 x0.630 DTW_selectiveTransitions
A path is not possible
A path is not possibleA path is not possible

A path is not possible
A path is not possible
Finished alignment for: train_toy x2.000 x0.630 DTW_selectiveTransitions
A path is not possibleA path is not possible

A path is not possible
A path is not possibleA path is not possible

Finished alignment for: train_toy x2.000 x0.500 DTW_selectiveTransitions
